# Creating training

## libraries

In [64]:
import numpy as np
import os
import tensorflow as tf

## Playing around with M00001

First, I start by bringing in "M00001.combo.bed".
This file contains all of the open chromatin regions for the motif M00001 for all tissue types.

Let's start by bringing the file in and seeing what it looks like first.

In [3]:
motifM00001 = []
with open("data/CENTIPEDEdata/motif.combo/M00001.combo.bed") as f:
    for line in f:
        motifM00001.append(line)

motifM00001[0]

'chr1\t74497\t74510\tM00001\t6.2881446601367\t- \n'

This brings each line as a long string

Instead, I would like to seperate out each column. I do this by adding the split() method. I have seen people use strip() and then a split() but I have yet to see any difference.

In [4]:
motifM00001 = []
with open("data/CENTIPEDEdata/motif.combo/M00001.combo.bed") as f:
    for line in f:
        motifM00001.append(line.split())

motifM00001[0]

['chr1', '74497', '74510', 'M00001', '6.2881446601367', '-']

Let's just grab the chr1 data for time saving.

Also, I really only want columns 2 and 3: the start and end positions for the motif.

In [5]:
motifM00001_chr1 = []
i = 0
while motifM00001[i][0] == "chr1":
    motifM00001_chr1.append(motifM00001[i][1:3]) # just grab index 1 and 2
    i += 1


print(motifM00001_chr1[-5:])
print(len(motifM00001))
print(len(motifM00001_chr1))

[['249099975', '249099988'], ['249100896', '249100909'], ['249108148', '249108161'], ['249209440', '249209453'], ['249236293', '249236306']]
65491
5182


In [6]:
region_length_dict = {}
for region in motifM00001_chr1:
    region_length = int(region[1]) - int(region[0])
    
    if region_length in region_length_dict:
        region_length_dict[region_length] += 1
    else:
        region_length_dict[region_length] = 1
        
region_length_dict

{13: 5182}

Here, we can see all region lengths are 13 because of the motif length.

Also, I would like to convert the elements into int

In [7]:
motifM00001_chr1 = [list(map(int,i)) for i in motifM00001_chr1]

In [8]:
motifM00001_chr1[-5:]

[[249099975, 249099988],
 [249100896, 249100909],
 [249108148, 249108161],
 [249209440, 249209453],
 [249236293, 249236306]]

## narrowPeak file
I do the same for the narrowPeak file, which contains all open regions for the LCL tissue type.

In [9]:
open_LCL_chr1 = []
with open("data/CENTIPEDEdata/wgEncodeAwgDnaseUwdukeGm12878UniPk.narrowPeak") as f:
    for line in f:
        if line.split()[0] == "chr1":
            open_LCL_chr1.append(line.split()[1:3])

open_LCL_chr1 = [list(map(int,i)) for i in open_LCL_chr1]
print(len(open_LCL_chr1))

17753


In [10]:
open_LCL_chr1[-5:]

[[249200725, 249200875],
 [249200920, 249201070],
 [249218985, 249219135],
 [249219525, 249219675],
 [249220645, 249220795]]

The list "narrowPeak_LCL_chr1" will be the building block for the "output" vector. Any regions that overlap with the motifM00001_chr1 list will be "1" in the input matrix for the column motifM00001_chr1. This is all probably be a lot easier if we use pandas instead now.

In [11]:
region_length_dict = {}
for region in open_LCL_chr1:
    region_length = int(region[1]) - int(region[0])
    
    if region_length in region_length_dict:
        region_length_dict[region_length] += 1
    else:
        region_length_dict[region_length] = 1
        
region_length_dict

{150: 17693, 250: 20, 270: 12, 290: 20, 230: 6, 210: 2}

We can see here that the region length is mostly 150 but not all.

The cell below is not helpful EXCEPT for learning about np.insert

## working with ClusteredV3.bed file

This is the file where all open chromatin regions for ALL tissues are stored. The tissues are identified by the numbers.

In [12]:
open_all_tissues_chr1 = []
with open("data/CENTIPEDEdata/wgEncodeRegDnaseClusteredV3.bed") as f:
    for line in f:
        if line.split()[0] == "chr1":
            open_all_tissues_chr1.append(line.split()[1:3])
            
open_all_tissues_chr1 = [list(map(int,i)) for i in open_all_tissues_chr1]
print(len(open_all_tissues_chr1))

173415


In [13]:
open_all_tissues_chr1[-5:]

[[249222400, 249222575],
 [249222960, 249223130],
 [249223340, 249223490],
 [249223825, 249223975],
 [249239705, 249239930]]

IMPORTANT TO NOTE HERE IS THAT THE REGION LENGTH CHANGES A LOT. press "O" to see. I have left the cell as "raw" because I don't want it to run all the time

## making output vector

We have all open regions for the LCL tissue. Now we would like some closed regions. We use the open regions in ALL tissues as candidates for closed regions for the LCL tissue. In other words we want to create output instances with "0".

Let's remind myself what they look like first

In [14]:
open_LCL_chr1[-5:]

[[249200725, 249200875],
 [249200920, 249201070],
 [249218985, 249219135],
 [249219525, 249219675],
 [249220645, 249220795]]

In [15]:
open_all_tissues_chr1[-5:]

[[249222400, 249222575],
 [249222960, 249223130],
 [249223340, 249223490],
 [249223825, 249223975],
 [249239705, 249239930]]

In [16]:
len(open_LCL_chr1)

17753

In [17]:
len(open_all_tissues_chr1)

173415

The below is unhelpful. I converted it into raw code cells.

#### below is old code

instead, do:

#### BEWARE: IF THE LAST REGION IN 'OPEN_LCL' IS PAST 'OPEN_ALL' THEN WE WOULD HAVE AN OUT OF BOUNDS ERROR. ALSO, THE LAST FEW ENTRIES OF 'OPEN_ALL' HAVE BEEN ADDED BUT IN A BANDAID WAY... NEED TO FIX

In [18]:
output = []
i = 0
for region in open_LCL_chr1:
    
    # while no overlap
    while open_all_tissues_chr1[i][1] < region[0]:
        output.append([open_all_tissues_chr1[i][0], open_all_tissues_chr1[i][1], 0])
        i += 1
    
    # now we must be at an overlap or already past 'region'
    
    # no overlap
    if open_all_tissues_chr1[i][0] > region[1]:
        # insert 'region'
        output.append([region[0], region[1], 1])
        
    else:
        # i MUST be an overlap now 

        # insert 'region'
        output.append([region[0], region[1], 1])
        
        # don't insert all_tissue regions until the overlap ends
        while (open_all_tissues_chr1[i][0] < region[1]):
            i += 1
            
# ENTRIES THAT HAVE NOT BEEN ADDED BUT SHOULD BE
for row in open_all_tissues_chr1[i:]:
    output.append([row[0], row[1], 0])

In [19]:
len(output)

176129

In [20]:
np.array(output)

array([[    10100,     10330,         0],
       [    10345,     10590,         0],
       [    16100,     16315,         0],
       ...,
       [249223340, 249223490,         0],
       [249223825, 249223975,         0],
       [249239705, 249239930,         0]])

In [21]:
np.sum(output, axis=0)[2]

17753

We can see from above that all open_LCL regions have been successfully added. 

In [22]:
17753/176122

0.10079944583868

around 10 percent of the output vector is classified as open.

Our final vector should be:

In [23]:
y_train = np.delete(output, [0,1], axis=1)
np.transpose(y_train)

array([[0, 0, 0, ..., 0, 0, 0]])

## input data

This brings us back to our M00001.bed file. Remember: this contains binding locations for motif M00001 in ALL tissues.

Our final goal is: ![](imgs/data.png)

In [24]:
motifM00001_chr1[-5:]

[[249099975, 249099988],
 [249100896, 249100909],
 [249108148, 249108161],
 [249209440, 249209453],
 [249236293, 249236306]]

In [25]:
regions = np.delete(output, 2, axis=1)
regions

array([[    10100,     10330],
       [    10345,     10590],
       [    16100,     16315],
       ...,
       [249223340, 249223490],
       [249223825, 249223975],
       [249239705, 249239930]])

In [53]:
motif_col = []
i=0
for region in regions:
    if i > len(motifM00001_chr1):
        break
    
    # if no overlap
    if region[1] < motifM00001_chr1[i][0]:
        motif_col.append(0)
    else:
        # no overlap
        if region[0] > motifM00001_chr1[i][1]:
            motif_col.append(0)
            i += 1 # region is past the motif[i], move forward
        
        # must be overlap
        else:
            motif_col.append(1)
            # don't move forward as the next region might also be overlap
            
np.sum(motif_col)

1804

seems to be quite sparse. might be a problem.

## multiple motif files

Now the challenge is getting many motif.bed files an putting them together. Let's try using os and keep all the motif files in one place to access them as I go instead of making a new array that carries all motif arrays.

In [30]:
path = "data/CENTIPEDEdata/motif.combo"
for motiffile in os.listdir(path):
    motif_combo = []
    with open(os.path.join(path, motiffile)) as f:
        for line in f:
            if line.split()[0] == "chr1":
                motif_combo.append(line.split()[1:3])
    
    motif_combo = [list(map(int,i)) for i in motif_combo]
    print(len(motif_combo))
    print(motif_combo[0:5])

5182
[[74497, 74510], [104986, 104999], [131373, 131386], [132173, 132186], [172225, 172238]]
632
[[11401, 11417], [852989, 853005], [855168, 855184], [868745, 868761], [876582, 876598]]
1246
[[1199051, 1199062], [1943092, 1943103], [2044095, 2044106], [2240050, 2240061], [2281759, 2281770]]
101
[[6830517, 6830534], [8610228, 8610245], [10347606, 10347623], [14883795, 14883812], [14964135, 14964152]]
1725
[[855947, 855963], [958589, 958605], [1005655, 1005671], [1026158, 1026174], [1079608, 1079624]]


okay we can very easily bring motif files in.

In [65]:
training_data = []
path = "data/CENTIPEDEdata/motif.combo"
for motiffile in os.listdir(path):
    motif_combo = []
    with open(os.path.join(path, motiffile)) as f:
        for line in f:
            if line.split()[0] == "chr1":
                motif_combo.append(line.split()[1:3])
    
    motif_combo = [list(map(int,i)) for i in motif_combo]

    motif_col = []
    i=0
    for region in regions:
        if i >= len(motif_combo):
            motif_col.append(0)
        else:
            # if no overlap
            if region[1] < motif_combo[i][0]:
                motif_col.append(0)
            else:
                # no overlap
                if region[0] > motif_combo[i][1]:
                    motif_col.append(0)
                    i += 1 # region is past the motif[i], move forward
        
                # must be overlap
                else:
                    motif_col.append(1)
                    # don't move forward as the next region might also be overlap
    
    print(len(motif_col))
    training_data.append(motif_col)
    

176129
176129
176129
176129
176129


In [66]:
x_train = np.transpose(training_data)

In [67]:
x_train

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [68]:
x_train.shape

(176129, 5)

In [69]:
np.sum(x_train, axis=0)

array([1804,  407,  165,   33,  907])

okay, we have something we can work with. The next step would be to do this for ALL motif files for ALL chromosomes as well and not just chr1. The attributes pos0 and pos1 in the files "motif.combo" are relative to the chromosome which may pose a challenge later. But for now, I think I can test to see if this format will work in a neural net.

## using a multilayer perceptron

Here, I use tensorflow and keras to build a simple neural net. I would expect it to perform pretty bad I think because of our sparseness and the fact that I haven't used everything yet.

In [75]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metris=['accuracy'])
model.fit(x_train, y_train, epochs=3)

model.summary()

Epoch 1/3
176129/176129 [==============================] - 23s 132us/sample - loss: 0.3291
Epoch 2/3
176129/176129 [==============================] - 22s 125us/sample - loss: 0.3271
Epoch 3/3
176129/176129 [==============================] - 23s 131us/sample - loss: 0.3269
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              multiple                  768       
_________________________________________________________________
dense_7 (Dense)              multiple                  16512     
_________________________________________________________________
dense_8 (Dense)              multiple                  258       
Total params: 17,538
Trainable params: 17,538
Non-trainable params: 0
_________________________________________________________________


Results aren't anything spectacular but as long as I know that this data preprocessing works on tensorflow, I am happy... for now.